In [35]:
import os
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [36]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
SCR_dir = os.path.join(project_dir, 'EDR')
fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')
plot_dir = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/plots/secondHalf')

In [37]:
def rename_roi(roi):
    cleaned_roi = roi.replace('_mask', '').replace('_mask', '')
    sides = ['lh', 'rh']
    regions = ['hippocampus', 'amygdala', 'vmpfc', 'insula', 'ACC']  # Add more as needed
    
    side = None
    region = None
    for s in sides:
        if s in cleaned_roi:
            side = s
            break
    for r in regions:
        if r in cleaned_roi:
            region = r
            break
    if side and region:
        return f"{side}_{region}"
    else:
        return cleaned_roi

In [38]:
# Define directories
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'

sub_no_MRI_phase2 = ['N102', 'N208']
sub_no_MRI_phase3 = ['N102', 'N208', 'N120']

SCR_dir = os.path.join(root_dir, project_name, 'EDR')
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')

# Load behavioral data
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

group_info_df_phase2 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase2)]
group_info_df_phase3 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase3)]

fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
df_fMRI['roi_name'] = df_fMRI['ROI'].apply(rename_roi)
# Define the desired order for 'roi_name'
roi_order = ['lh_ACC', 'rh_ACC', 
             'lh_vmpfc','rh_vmpfc',
             'lh_insula','rh_insula', 
             'lh_hippocampus', 'rh_hippocampus', 
             'lh_amygdala', 'rh_amygdala'
]

df_fMRI['roi_name'] = pd.Categorical(df_fMRI['roi_name'], categories=roi_order, ordered=True)

df_fMRI_phase2 = df_fMRI.loc[df_fMRI['Task']=='phase2'].copy()
df_fMRI_phase3 = df_fMRI.loc[df_fMRI['Task']=='phase3'].copy()

In [39]:
df_fMRI_phase2['subID'] = df_fMRI_phase2['Subject'].apply(lambda x: group_info_df_phase2.iloc[x - 1]['subID'])
df_fMRI_phase3['subID'] = df_fMRI_phase3['Subject'].apply(lambda x: group_info_df_phase3.iloc[x - 1]['subID'])
df_phase2 = df_fMRI_phase2.merge(group_info_df_phase2, how='left', left_on='subID', right_on='subID')
df_phase2 = df_phase2[df_phase2['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)
df_phase3 = df_fMRI_phase3.merge(group_info_df_phase3, how='left', left_on='subID', right_on='subID')
df_phase3 = df_phase3[df_phase3['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)

In [40]:
df_current = df_phase2

In [41]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        df_tmp = df_current.loc[(df_current['roi_name'] == roi) &
                                (df_current['group'] == group) &
                                (df_current['Contrast'].isin([20,22]))].copy()
        df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
        t_statistic, p_value = stats.ttest_rel(df_pivot[[20]], df_pivot[[22]])
        results.append({
            'roi': roi,
            'group': group,
            't_stat': t_statistic[0],
            'p_value': p_value[0],
            'n': len(df_pivot)  # Sample size
        })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,group,t_stat,p_value,n,p_value_corrected
11,lh_insula,Controls,3.111848,0.003044,52,0.060877
3,rh_ACC,Controls,2.515079,0.015094,52,0.301887
0,lh_hippocampus,Patients,-0.169120,0.866413,49,1.000000
17,lh_vmpfc,Controls,1.715717,0.092284,52,1.000000
16,lh_vmpfc,Patients,-1.261082,0.213374,49,1.000000
15,rh_insula,Controls,0.339114,0.735917,52,1.000000
14,rh_insula,Patients,0.906237,0.369337,49,1.000000
13,rh_amygdala,Controls,1.537827,0.130273,52,1.000000
12,rh_amygdala,Patients,-0.044177,0.964947,49,1.000000
10,lh_insula,Patients,-0.764680,0.448205,49,1.000000


In [42]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        for gender in df_current['Gender'].unique():
            for drug in df_current['Drug'].unique():
                df_tmp = df_current.loc[(df_current['roi_name'] == roi) & 
                                        (df_current['group'] == group) & 
                                        (df_current['Gender'] == gender) &
                                        (df_current['Drug'] == drug) &
                                        (df_current['Contrast'].isin([20,22]))].copy()
                df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
                t_statistic, p_value = stats.ttest_rel(df_pivot[[20]], df_pivot[[22]])
                results.append({
                'roi': roi,
                'Group': group,
                'Drug': drug,
                'Gender': gender,
                't_stat': t_statistic[0],
                'p_value': p_value[0],
                'n': len(df_pivot)  # Sample size
            })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,Group,Drug,Gender,t_stat,p_value,n,p_value_corrected
12,rh_ACC,Controls,Placebo,Female,3.946269,0.001673,14,0.133863
49,rh_amygdala,Patients,Oxytocin,Female,3.071723,0.007752,16,0.620171
36,rh_vmpfc,Controls,Placebo,Female,2.929324,0.011729,14,0.938287
0,lh_hippocampus,Patients,Placebo,Female,0.983544,0.344755,13,1.000000
57,rh_insula,Patients,Oxytocin,Female,-1.021120,0.323385,16,1.000000
...,...,...,...,...,...,...,...,...
24,lh_ACC,Patients,Placebo,Female,-0.275065,0.787945,13,1.000000
23,lh_amygdala,Controls,Oxytocin,Male,1.321962,0.213012,12,1.000000
22,lh_amygdala,Controls,Placebo,Male,0.235008,0.817864,14,1.000000
29,lh_ACC,Controls,Oxytocin,Female,0.684414,0.507877,12,1.000000


In [43]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([20,22]))].copy()
    df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                 -0.028
Method:                 Least Squares   F-statistic:                    0.6964
Date:                Wed, 19 Mar 2025   Prob (F-statistic):              0.710
Time:                        23:01:19   Log-Likelihood:                -139.95
No. Observations:                 101   AIC:                             299.9
Df Residuals:                      91   BIC:                             326.0
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
64,rh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.004344
67,rh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.005014
88,lh_vmpfc,Avoidance_centered,0.057232
65,rh_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],0.065654
19,rh_ACC,demo_age,0.069634
...,...,...,...
78,rh_insula,Avoidance_centered,0.960314
17,rh_ACC,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.966529
29,lh_amygdala,demo_age,0.971556
24,lh_amygdala,C(group)[T.Patients]:C(Drug)[T.Placebo],0.985751


for the entire dataset
group * drug in rh-amy;
group * drug * gender in rh-amy;
group * gender in rh-amy (marginal)
avoidance: lh-vmpfc (marginal)
age: r-acc

In [44]:
# extinction in only placebo
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin([20,22]))].copy()
    df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI     
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'ROI': roi,
            'Effect': effect_idx,
            'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
        })
    
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                 -0.059
Method:                 Least Squares   F-statistic:                    0.4524
Date:                Wed, 19 Mar 2025   Prob (F-statistic):              0.809
Time:                        23:01:22   Log-Likelihood:                -69.694
No. Observations:                  50   AIC:                             151.4
Df Residuals:                      44   BIC:                             162.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

,ROI,Effect,b,SE,t,p,CI_lower,CI_upper
37,rh_amygdala,C(group)[T.Patients],1.066794,0.408024,2.614540,0.012189,0.244476,1.889112
11,rh_ACC,demo_age,0.051937,0.020383,2.548048,0.014401,0.010858,0.093016
20,lh_ACC,C(Gender)[T.Male],0.945083,0.390043,2.423023,0.019575,0.159003,1.731163
39,rh_amygdala,C(group)[T.Patients]:C(Gender)[T.Male],-1.276500,0.578657,-2.205970,0.032654,-2.442706,-0.110293
25,rh_vmpfc,C(group)[T.Patients],0.730732,0.423110,1.727049,0.091174,-0.121991,1.583454
8,rh_ACC,C(Gender)[T.Male],0.585646,0.365990,1.600171,0.116717,-0.151958,1.323249
7,rh_ACC,C(group)[T.Patients],0.622289,0.389263,1.598634,0.117059,-0.162219,1.406796
26,rh_vmpfc,C(Gender)[T.Male],0.622253,0.397813,1.564183,0.124939,-0.179487,1.423993
31,lh_insula,C(group)[T.Patients],0.621350,0.412146,1.507597,0.138805,-0.209276,1.451977
35,lh_insula,demo_age,0.032100,0.021581,1.487425,0.144034,-0.011394,0.075595


In placebo. \
Group: rh-amy (rh_vmpfc marginal) \
Gender: lh-ACC \
Age: rh-ACC \
Group * Gender: rh-AMY

In [45]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([20,22]))].copy()
df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                col='group',
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_group.png', bbox_inches='tight')
plt.close()
#plt.show()

In [46]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([20,22]))].copy()
df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [47]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([20,22]))].copy()
df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='group', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_group_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [48]:
# extinction in each group
results_list = []
drug_results_list = []
for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {group}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                          (df_current['group'] == group) & 
                          (df_current['Contrast'].isin([20,22]))].copy()
        
        df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                        columns='CS_Type', 
                        values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
        #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        
        combined_col = f'{roi}_{group}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
        conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
        for effect_idx in result_ols.params.index:
            if effect_idx == 'C(Drug)[T.Placebo]': 
                drug_results_list.append({
                    'Effect': effect_idx,
                    'group': group,
                    'ROI': roi,
                    'b': result_ols.params[effect_idx],  # Coefficient
                    'SE': result_ols.bse[effect_idx],    # Standard error
                    't': result_ols.tvalues[effect_idx], # t-statistic
                    'p': result_ols.pvalues[effect_idx],  # p-value
                    'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                    'CI_upper': conf_int.loc[effect_idx, 1]
                })
    
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('p', ascending=True)
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Patients
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     2.174
Date:                Wed, 19 Mar 2025   Prob (F-statistic):             0.0747
Time:                        23:01:35   Log-Likelihood:                -64.006
No. Observations:                  49   AIC:                             140.0
Df Residuals:                      43   BIC:                             151.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

,Condition,Effect,p_value
39,rh_amygdala_Patients,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.000323
38,rh_amygdala_Patients,C(Gender)[T.Male],0.001621
37,rh_amygdala_Patients,C(Drug)[T.Placebo],0.004913
52,lh_vmpfc_Patients,Avoidance_centered,0.005447
64,lh_hippocampus_Controls,Avoidance_centered,0.014931
...,...,...,...
99,rh_amygdala_Controls,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.970622
40,rh_amygdala_Patients,Avoidance_centered,0.971976
82,lh_ACC_Controls,Avoidance_centered,0.981840
87,rh_vmpfc_Controls,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.986798


in group specific dataset, \
for Patients: \
    drug effect, gender effect and drug * gender was significant in rh_AMY; 
    gender in bi-insula \
    drug in lh-hip \
    drug * gender in rh-hip, rh-insula\
    avoidance in lh-vmpfc, lh-hip \
for Controls: \
    avoidance in lh-hip \
    

In [49]:
import pandas as pd
from scipy.stats import zscore, ttest_ind

desired_comparisons = [
    'Female_Oxytocin vs. Female_Placebo',
    'Male_Oxytocin vs. Male_Placebo'
]

t_test_results_list = []

for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                           (df_current['group'] == group) & 
                           (df_current['Contrast'].isin([20, 22]))].copy()
        
        df_tmp.loc[df_tmp['Contrast'] == 20, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast'] == 22, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID', 'Drug', 'Gender', 'group', 'Avoidance', 'demo_age'], 
                          columns='CS_Type', values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])  # Offset of 1 assumed intentional
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
        
        group_labels = df['Gender'].astype(str) + "_" + df['Drug'].astype(str)
        
        for comparison in desired_comparisons:
            g1, g2 = comparison.split(' vs. ')
            if g1 in group_labels.values and g2 in group_labels.values:  # Check if both groups exist
                group1_data = df[group_labels == g1]['z_fMRI']
                group2_data = df[group_labels == g2]['z_fMRI']
                
                t_stat, p_value = ttest_ind(group1_data, group2_data, nan_policy='omit')
                
                mean_diff = group1_data.mean() - group2_data.mean()
                se = ((group1_data.var() / group1_data.count()) + (group2_data.var() / group2_data.count())) ** 0.5
                
                t_test_results_list.append({
                    'ROI': roi,
                    'Comparison': comparison,
                    'Mean_Diff': mean_diff,
                    'SE': se,
                    't': t_stat,
                    'p': p_value
                })

# Create DataFrame with all results
df_t_test = pd.DataFrame(t_test_results_list)
df_t_test = df_t_test.sort_values('p', ascending=True)  # Sort by p-value

In [50]:
df_t_test

,ROI,Comparison,Mean_Diff,SE,t,p
12,rh_amygdala,Female_Oxytocin vs. Female_Placebo,-1.048838,0.311774,-3.427612,0.001967
13,rh_amygdala,Male_Oxytocin vs. Male_Placebo,1.039326,0.431955,2.399836,0.027436
19,rh_hippocampus,Male_Oxytocin vs. Male_Placebo,0.793007,0.398914,2.053835,0.054811
22,rh_ACC,Female_Oxytocin vs. Female_Placebo,0.627901,0.373191,1.720124,0.098281
0,lh_hippocampus,Female_Oxytocin vs. Female_Placebo,0.622134,0.379194,1.616761,0.117555
14,rh_insula,Female_Oxytocin vs. Female_Placebo,0.571399,0.371973,1.510529,0.142524
18,rh_hippocampus,Female_Oxytocin vs. Female_Placebo,-0.544266,0.406053,-1.380123,0.178870
1,lh_hippocampus,Male_Oxytocin vs. Male_Placebo,0.573845,0.432255,1.368523,0.187988
15,rh_insula,Male_Oxytocin vs. Male_Placebo,-0.494327,0.428296,-1.161801,0.260491
34,rh_insula,Female_Oxytocin vs. Female_Placebo,0.465985,0.431772,1.118634,0.274368


In [51]:
results_list = []
# extinction, in each gender
for g in df_current['Gender'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {g}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Gender'] == g) & (df_current['Contrast'].isin([20,22]))].copy()
        df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(group) * C(Drug) + Avoidance_centered + demo_age", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        combined_col = f'{roi}_{g}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Gender': g,
                'Effect': effect_idx,
                'p_value': p_value
            })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                 -0.015
Method:                 Least Squares   F-statistic:                    0.8445
Date:                Wed, 19 Mar 2025   Prob (F-statistic):              0.525
Time:                        23:01:42   Log-Likelihood:                -75.768
No. Observations:                  55   AIC:                             163.5
Df Residuals:                      49   BIC:                             175.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

,Condition,Gender,Effect,p_value
39,rh_amygdala_Female,Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.003824
52,lh_vmpfc_Female,Female,Avoidance_centered,0.050342
53,lh_vmpfc_Female,Female,demo_age,0.052354
103,rh_insula_Male,Male,C(group)[T.Patients],0.062506
117,rh_hippocampus_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.075903
...,...,...,...,...
119,rh_hippocampus_Male,Male,demo_age,0.956883
28,rh_vmpfc_Female,Female,Avoidance_centered,0.960331
81,lh_ACC_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.979086
68,rh_ACC_Male,Male,C(Drug)[T.Placebo],0.990632


in Gender specific data subset, 
for Female: \
    group * drug in rh-amy \
    avoidance and age in lh_vmpfc \
for Male (marginal): \
    group in rh-insula \
    group * drug and avoidance in rh-hip

In [52]:
import pandas as pd 
from statsmodels.formula.api import ols
from scipy.stats import zscore

results_list = []
for g in df_current['Gender'].unique():
    for group in df_current['group'].unique():
        for roi in df_current['roi_name'].unique():
            print(f'extinction results for {roi} {g}')
            df_tmp = df_current[(df_current['roi_name'] == roi) & 
                              (df_current['Gender'] == g) & 
                              (df_current['group'] == group) & 
                              (df_current['Contrast'].isin([20,22]))].copy()
            
            df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
            df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
            
            df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                            columns='CS_Type', 
                            values='Mean_COPE_Value').reset_index()
            
            df['fMRI'] = df['CSR'] - df['CSS']
            df['z_fMRI'] = 1 + zscore(df['fMRI'])
            df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
            df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
            
            df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
            
            model_ols = ols("z_fMRI ~ C(Drug) + Avoidance_centered + demo_age", data=df)
            
            #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
            result_ols = model_ols.fit()
            print(result_ols.summary())
            
            combined_col = f'{roi}_{group}_{g}'
            
            for effect_idx in result_ols.pvalues.index:
                p_value = result_ols.pvalues[effect_idx]
                results_list.append({
                    'Gender': g,
                    'Group': group,
                    'Condition': combined_col,
                    'Effect': effect_idx,
                    'p_value': p_value
                })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.115
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.078
Date:                Wed, 19 Mar 2025   Prob (F-statistic):              0.376
Time:                        23:01:46   Log-Likelihood:                -39.385
No. Observations:                  29   AIC:                             86.77
Df Residuals:                      25   BIC:                             92.24
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

,Gender,Group,Condition,Effect,p_value
25,Female,Patients,rh_amygdala_Patients_Female,C(Drug)[T.Placebo],0.005352
82,Male,Patients,lh_hippocampus_Patients_Male,Avoidance_centered,0.007650
99,Male,Patients,rh_vmpfc_Patients_Male,demo_age,0.019652
81,Male,Patients,lh_hippocampus_Patients_Male,C(Drug)[T.Placebo],0.020733
34,Female,Patients,lh_vmpfc_Patients_Female,Avoidance_centered,0.026467
...,...,...,...,...,...
106,Male,Patients,rh_amygdala_Patients_Male,Avoidance_centered,0.956076
147,Male,Controls,rh_amygdala_Controls_Male,demo_age,0.970972
90,Male,Patients,lh_amygdala_Patients_Male,Avoidance_centered,0.974858
119,Male,Patients,rh_hippocampus_Patients_Male,demo_age,0.976855


in group * gender specific data subset: \
drug effect in rh-amy for Female-Patients \
avoidance in lh-vmpfc for Female-Patients \
drug in bi-hip and rh-amy for Male-Patients \
and avoidance in lh-hip and lh-ACC for Male-Patients \
age in rh-vmpfc in Male-Patients

In [53]:
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([20, 22]))].copy()
    df_tmp.loc[df_tmp['Contrast'] == 20, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast'] == 22, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Drug', y='z_fMRI', hue='CS_Type',
                    order=['Placebo', 'Oxytocin'], 
                    col='group', 
                    row='Gender', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in Extinction', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'{plot_dir}/extinction_plot_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

lh_hippocampus
rh_ACC
lh_amygdala
lh_ACC
rh_vmpfc
lh_insula
rh_amygdala
rh_insula
lh_vmpfc
rh_hippocampus
